In [1]:
import pandas as pd
import numpy as np
import keras
import scipy
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from sklearn.model_selection import GroupKFold
from sklearn import linear_model
from lineartree import RandomForestRegressor
from sklearn import metrics
from scipy.signal import savgol_filter
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [2]:
from numpy.random import seed
seed(1)
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [3]:
def rae(actual, predicted):
    actual = np.asarray(actual).reshape(-1,1)
    predicted = np.asarray(predicted).reshape(-1,1)
    numerator = np.sum(np.abs(predicted - actual))
    denominator = np.sum(np.abs(np.mean(actual) - actual))
    return numerator / denominator

In [4]:
def peak_eV_nm(peak_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    return 1239.84193/peak_eV

In [5]:
def FWHM_eV_nm(peak_eV,fwhm_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    fwhm_eV = np.asarray(fwhm_eV).reshape(-1,1)
    return  2*(1239.84193/(peak_eV-fwhm_eV/2) - 1239.84193/(peak_eV))

In [6]:
def predict_dist(X, model, num_samples):
    preds = [model(X, training=True) for _ in range(num_samples)]
    return np.hstack(preds)

In [7]:
def predict_point(X, model, num_samples):
    pred_dist = predict_dist(X, model, num_samples)
    return pred_dist.mean(axis=1)

In [8]:
#Define models

initializer = keras.initializers.HeNormal(seed=1)
opt = keras.optimizers.Adam(learning_rate=0.001)

model_peak = keras.models.Sequential()
model_peak.add(Dense(5, input_shape=(5,), kernel_initializer=initializer, activation='relu'))
model_peak.add(Dense(50,  activation='relu'))
model_peak.add(keras.layers.Dropout(0.2))
model_peak.add(Dense(50,  activation='relu'))
model_peak.add(keras.layers.Dropout(0.2))
model_peak.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_peak.compile(loss='mean_squared_error', optimizer= opt)

model_fwhm = keras.models.Sequential()
model_fwhm.add(Dense(5, input_shape=(5,), kernel_initializer=initializer, activation='relu'))
model_fwhm.add(Dense(50,  activation='relu'))
model_fwhm.add(keras.layers.Dropout(0.2))
model_fwhm.add(Dense(50,  activation='relu'))
model_fwhm.add(keras.layers.Dropout(0.2))
model_fwhm.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_fwhm.compile(loss='mean_squared_error', optimizer= opt)

model_cascade = keras.models.Sequential()
model_cascade.add(Dense(6, input_shape=(6,), kernel_initializer=initializer, activation='relu'))
model_cascade.add(Dense(50,  activation='relu'))
model_cascade.add(keras.layers.Dropout(0.2))
model_cascade.add(Dense(50,  activation='relu'))
model_cascade.add(keras.layers.Dropout(0.2))
model_cascade.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_cascade.compile(loss='mean_squared_error', optimizer= opt)


In [9]:
#Xval on Train/Val set, 10-fold cross validation (Group splits), PL Peak, FWHM (eV), Normalized
# load dataset into Pandas DataFrame (cleaned PL data)
df = pd.read_csv('CdSe PL Train Val.csv')

#normalize input and output data
min_params = pd.Series([0, 0, 3, 200, 0, 400, 15, 1239.84193/620, 0.05], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
max_params = pd.Series([50, 50, 150, 310, 600, 620, 80, 1239.84193/400, 0.4], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] = (df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] - min_params) / (max_params- min_params)

X = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s']]
X2 = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s','Peak_eV']]
y = df['Peak_eV']
z = df['FWHM_eV']
group = df['Expt. No']

#Split groups for cross validation by Experiment number
gkf = GroupKFold(n_splits=10)
matrix = np.zeros([10,16])
matrix_fwhm = np.zeros([10,16])
matrix_cascade = np.zeros([10,16])

j = 0
#Loop through the 10 splits
for train_index, test_index in gkf.split(X, groups=group):
    #Separate out training and test sets
    X_train = X.iloc[train_index, :]
    X2_train = X2.iloc[train_index, :]
    y_train = y[train_index]
    z_train = z[train_index]
    X_test = X.iloc[test_index, :]
    X2_test = X2.iloc[test_index, :]
    y_test = y[test_index]
    z_test = z[test_index]
    
    #Load weights
    model_peak.load_weights('initialize_weights3.h5')
    model_fwhm.load_weights('initialize_weights3.h5')
    model_cascade.load_weights('initialize_weights4.h5')

    #Train models
    #prediction for Peak
    model_peak.fit(X_train, y_train, epochs=300, validation_data=(X_test, y_test),verbose = 1, use_multiprocessing=False, workers = 100)
    y_pred = model_peak.predict(X_test)
    #prediction for FWHM (direct)
    model_fwhm.fit(X_train, z_train, epochs=300, validation_data=(X_test, z_test),verbose = 1, use_multiprocessing=False, workers = 100)
    y_pred_fwhm = model_fwhm.predict(X_test)
    #prediction for FWHM (cascade)
    model_cascade.fit(X2_train, z_train, epochs=300, validation_data=(X2_test, z_test),verbose = 1, use_multiprocessing=False, workers = 100)
    X_test.insert(5, "Peak_eV", model_peak.predict(X_test), True)
    y_pred_cascade = model_cascade.predict(X_test)

    
    #prediction for Peak with dropout
    X_test = X.iloc[test_index, :]
    X_test_t = K.constant(X_test)
    y_pred_d =  predict_point(X_test_t, model_peak, 1000)
    #prediction for FWHM (direct)
    y_pred_fwhm_d = predict_point(X_test_t, model_fwhm, 1000)
    #prediction for FWHM (cascade)
    X_test.insert(5, "Peak_eV", y_pred_d, True)
    X_test_t2 = K.constant(X_test)
    y_pred_cascade_d = predict_point(X_test_t2, model_cascade, 1000)
    
    #errors in eV (normalized)
    #UN-normalize eV predictions by scaling with ratio

    ratio = (1239.84193/400 - 1239.84193/620)
    ratio2 = 0.35
    matrix[j,0] = metrics.mean_absolute_error(y_test, y_pred)*ratio
    matrix[j,1] = metrics.r2_score(y_test, y_pred)
    matrix[j,2] = rae(y_test, y_pred)
    matrix[j,3] = ratio*np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    matrix[j,4] = metrics.mean_absolute_error(y_test, y_pred_d)*ratio 
    matrix[j,5] = metrics.r2_score(y_test, y_pred_d)
    matrix[j,6] = rae(y_test, y_pred_d)
    matrix[j,7] = ratio*np.sqrt(metrics.mean_squared_error(y_test, y_pred_d))

    matrix_fwhm[j,0] = metrics.mean_absolute_error(z_test, y_pred_fwhm)*ratio2
    matrix_fwhm[j,1] = metrics.r2_score(z_test, y_pred_fwhm)
    matrix_fwhm[j,2] = rae(z_test, y_pred_fwhm)
    matrix_fwhm[j,3] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_fwhm))
    matrix_fwhm[j,4] = metrics.mean_absolute_error(z_test, y_pred_fwhm_d)*ratio2
    matrix_fwhm[j,5] = metrics.r2_score(z_test, y_pred_fwhm_d)
    matrix_fwhm[j,6] = rae(z_test, y_pred_fwhm_d)
    matrix_fwhm[j,7] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_fwhm_d))
    
    matrix_cascade[j,0] = metrics.mean_absolute_error(z_test, y_pred_cascade)*ratio2
    matrix_cascade[j,1] = metrics.r2_score(z_test, y_pred_cascade)
    matrix_cascade[j,2] = rae(z_test, y_pred_cascade)
    matrix_cascade[j,3] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_cascade))
    matrix_cascade[j,4] = metrics.mean_absolute_error(z_test, y_pred_cascade_d)*ratio2
    matrix_cascade[j,5] = metrics.r2_score(z_test, y_pred_cascade_d)
    matrix_cascade[j,6] = rae(z_test, y_pred_cascade_d)
    matrix_cascade[j,7] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_cascade_d))
    
    

    #errors in nm
    #UNnormalize the data first
    y_test = y_test*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    y_pred = y_pred*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    y_pred_d = y_pred_d*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    z_test = z_test*(0.35)+0.05
    y_pred_fwhm = y_pred_fwhm*(0.35)+0.05
    y_pred_fwhm_d = y_pred_fwhm_d*(0.35)+0.05
    y_pred_cascade = y_pred_cascade*(0.35)+0.05
    y_pred_cascade_d = y_pred_cascade_d*(0.35)+0.05
    
    #Calculate data in nm and eV
    matrix[j,8] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,9] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,10] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,11] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred)))
    matrix[j,12] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,13] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,14] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,15] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred_d)))

    matrix_fwhm[j,8] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,9] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,10] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,11] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm)))
    matrix_fwhm[j,12] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d)) 
    matrix_fwhm[j,13] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d))
    matrix_fwhm[j,14] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d)) 
    matrix_fwhm[j,15] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d))) 

    matrix_cascade[j,8] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,9] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,10] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,11] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade)))
    matrix_cascade[j,12] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,13] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,14] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,15] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d)))

    j+=1

matrix_2 = np.zeros([1,32])
matrix_2[0,:16] = matrix.mean(0)[:16]
matrix_2[0,16:32] = np.std(matrix[:,:16], axis=0)

matrix_3 = np.zeros([1,32])
matrix_3[0,:16] = matrix_fwhm.mean(0)[:16]
matrix_3[0,16:32]= np.std(matrix_fwhm[:,:16], axis=0)

matrix_4 = np.zeros([1,32])
matrix_4[0,:16] = matrix_cascade.mean(0)[:16]
matrix_4[0,16:32] = np.std(matrix_cascade[:,:16], axis=0)

np.set_printoptions(precision=5, threshold=5, edgeitems=4, suppress=True)


Epoch 1/300
72/72 [==============================] - 1s 4ms/step - loss: 0.0414 - val_loss: 0.0204
Epoch 2/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0138 - val_loss: 0.0161
Epoch 3/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0100 - val_loss: 0.0125
Epoch 4/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0084 - val_loss: 0.0098
Epoch 5/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0072 - val_loss: 0.0079
Epoch 6/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0062 - val_loss: 0.0067
Epoch 7/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0058 - val_loss: 0.0066
Epoch 8/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0053 - val_loss: 0.0052
Epoch 9/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0050 - val_loss: 0.0053
Epoch 10/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0048 - val_loss: 0.0043
Epoch 11/

72/72 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 84/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0014
Epoch 85/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0015
Epoch 86/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 87/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0015
Epoch 88/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 89/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 90/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 91/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 92/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 0.0013
Epoch 93/300


72/72 [==============================] - 0s 2ms/step - loss: 0.0010 - val_loss: 9.8435e-04
Epoch 165/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0010 - val_loss: 9.5635e-04
Epoch 166/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 167/300
72/72 [==============================] - 0s 2ms/step - loss: 9.7122e-04 - val_loss: 0.0013
Epoch 168/300
72/72 [==============================] - 0s 2ms/step - loss: 9.0826e-04 - val_loss: 0.0011
Epoch 169/300
72/72 [==============================] - 0s 2ms/step - loss: 9.6541e-04 - val_loss: 9.8584e-04
Epoch 170/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0010 - val_loss: 0.0011
Epoch 171/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0010 - val_loss: 0.0012
Epoch 172/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0010 - val_loss: 0.0011
Epoch 173/300
72/72 [==============================] - 0s 2ms/step - loss: 9.7785

Epoch 242/300
72/72 [==============================] - 0s 2ms/step - loss: 9.4867e-04 - val_loss: 0.0011
Epoch 243/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0010 - val_loss: 0.0011
Epoch 244/300
72/72 [==============================] - 0s 2ms/step - loss: 9.4486e-04 - val_loss: 0.0010
Epoch 245/300
72/72 [==============================] - 0s 2ms/step - loss: 9.4230e-04 - val_loss: 8.9140e-04
Epoch 246/300
72/72 [==============================] - 0s 1ms/step - loss: 9.2167e-04 - val_loss: 9.5388e-04
Epoch 247/300
72/72 [==============================] - 0s 2ms/step - loss: 9.8514e-04 - val_loss: 0.0011
Epoch 248/300
72/72 [==============================] - 0s 2ms/step - loss: 9.6394e-04 - val_loss: 9.8703e-04
Epoch 249/300
72/72 [==============================] - 0s 1ms/step - loss: 9.1086e-04 - val_loss: 8.8717e-04
Epoch 250/300
72/72 [==============================] - 0s 2ms/step - loss: 8.5474e-04 - val_loss: 8.6786e-04
Epoch 251/300
72/72 [==================

72/72 [==============================] - 0s 2ms/step - loss: 0.0022 - val_loss: 0.0033
Epoch 19/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0022 - val_loss: 0.0031
Epoch 20/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0022 - val_loss: 0.0031
Epoch 21/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0021 - val_loss: 0.0030
Epoch 22/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0022 - val_loss: 0.0030
Epoch 23/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0022 - val_loss: 0.0030
Epoch 24/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0021 - val_loss: 0.0032
Epoch 25/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0021 - val_loss: 0.0031
Epoch 26/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0021 - val_loss: 0.0029
Epoch 27/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0020 - val_loss: 0.0030
Epoch 28/300


...

72/72 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 8.6801e-04
Epoch 248/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 8.8819e-04
Epoch 249/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 8.5078e-04
Epoch 250/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 8.4329e-04
Epoch 251/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 8.6627e-04
Epoch 252/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 8.2559e-04
Epoch 253/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 8.4839e-04
Epoch 254/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 8.3787e-04
Epoch 255/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 8.2042e-04
Epoch 256/300
72/72 [==============================] - 0s 1ms/step - 

Epoch 26/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0020 - val_loss: 0.0011
Epoch 27/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0020 - val_loss: 0.0010
Epoch 28/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0021 - val_loss: 0.0011
Epoch 29/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0020 - val_loss: 9.9409e-04
Epoch 30/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0020 - val_loss: 0.0010
Epoch 31/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0020 - val_loss: 9.7898e-04
Epoch 32/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0020 - val_loss: 0.0010
Epoch 33/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0019 - val_loss: 9.9578e-04
Epoch 34/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0019 - val_loss: 9.8756e-04
Epoch 35/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0019 - v

72/72 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 9.3201e-04
Epoch 106/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 8.9151e-04
Epoch 107/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 8.6937e-04
Epoch 108/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 8.8082e-04
Epoch 109/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0010
Epoch 110/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 9.5527e-04
Epoch 111/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 8.4510e-04
Epoch 112/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 8.5850e-04
Epoch 113/300
72/72 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 9.4703e-04
Epoch 114/300
72/72 [==============================] - 0s 1ms/step - loss

72/72 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 9.0125e-04
Epoch 184/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 8.1968e-04
Epoch 185/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 9.3310e-04
Epoch 186/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 9.2391e-04
Epoch 187/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 8.2368e-04
Epoch 188/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 9.3886e-04
Epoch 189/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 8.4097e-04
Epoch 190/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 9.1223e-04
Epoch 191/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 8.4269e-04
Epoch 192/300
72/72 [==============================] - 0s 1ms/step - 

72/72 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 7.6510e-04
Epoch 262/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 8.5126e-04
Epoch 263/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 8.2195e-04
Epoch 264/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 7.8051e-04
Epoch 265/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 8.7077e-04
Epoch 266/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 8.3596e-04
Epoch 267/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 7.6789e-04
Epoch 268/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 9.2351e-04
Epoch 269/300
72/72 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 8.5642e-04
Epoch 270/300
72/72 [==============================] - 0s 1ms/step - 

In [10]:
print('\033[1m' + 'Model without Dropout at Inference' + '\033[0m')
print('\033[1m' + 'Absorbance Peak' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_2[0,0]), "{:.4f}".format(matrix_2[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_2[0,1]), "{:.4f}".format(matrix_2[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_2[0,2]), "{:.4f}".format(matrix_2[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_2[0,3]), "{:.4f}".format(matrix_2[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_2[0,8]), "{:.4f}".format(matrix_2[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_2[0,9]), "{:.4f}".format(matrix_2[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_2[0,10]), "{:.4f}".format(matrix_2[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_2[0,11]), "{:.4f}".format(matrix_2[0,27]))

print('\033[1m' + 'Absorbance FWHM (direct)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_3[0,0]), "{:.4f}".format(matrix_3[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_3[0,1]), "{:.4f}".format(matrix_3[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_3[0,2]), "{:.4f}".format(matrix_3[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_3[0,3]), "{:.4f}".format(matrix_3[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_3[0,8]), "{:.4f}".format(matrix_3[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_3[0,9]), "{:.4f}".format(matrix_3[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_3[0,10]), "{:.4f}".format(matrix_3[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_3[0,11]), "{:.4f}".format(matrix_3[0,27]))

print('\033[1m' + 'Absorbance FWHM (cascade)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_4[0,0]), "{:.4f}".format(matrix_4[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_4[0,1]), "{:.4f}".format(matrix_4[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_4[0,2]), "{:.4f}".format(matrix_4[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_4[0,3]), "{:.4f}".format(matrix_4[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_4[0,8]), "{:.4f}".format(matrix_4[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_4[0,9]), "{:.4f}".format(matrix_4[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_4[0,10]), "{:.4f}".format(matrix_4[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_4[0,11]), "{:.4f}".format(matrix_4[0,27]))

Model without Dropout at Inference
Absorbance Peak
MAE (eV): 	 0.0235 0.0035
R2 (eV):	 0.9599 0.0136
RAE (eV):	 0.1798 0.0289
RMSE (eV):	 0.0305 0.0047
MAE (nm):	 5.3390 0.7775
R2 (nm):	 0.9586 0.0124
RAE (nm):	 0.1821 0.0274
RMSE (nm):	 6.9127 0.9829
Absorbance FWHM (direct)
MAE (eV): 	 0.0107 0.0016
R2 (eV):	 0.4215 0.1449
RAE (eV):	 0.6624 0.0563
RMSE (eV):	 0.0158 0.0027
MAE (nm):	 2.6568 0.4405
R2 (nm):	 0.5400 0.1543
RAE (nm):	 0.5794 0.0617
RMSE (nm):	 3.9204 0.8171
Absorbance FWHM (cascade)
MAE (eV): 	 0.0087 0.0014
R2 (eV):	 0.5768 0.1141
RAE (eV):	 0.5366 0.0642
RMSE (eV):	 0.0134 0.0024
MAE (nm):	 2.1463 0.3449
R2 (nm):	 0.6858 0.0917
RAE (nm):	 0.4703 0.0614
RMSE (nm):	 3.2229 0.5789


In [11]:
print('\033[1m' + 'Model with Dropout at Inference' + '\033[0m')
print('\033[1m' + 'Absorbance Peak' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_2[0,4]), "{:.4f}".format(matrix_2[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_2[0,5]), "{:.4f}".format(matrix_2[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_2[0,6]), "{:.4f}".format(matrix_2[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_2[0,7]), "{:.4f}".format(matrix_2[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_2[0,12]), "{:.4f}".format(matrix_2[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_2[0,13]), "{:.4f}".format(matrix_2[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_2[0,14]), "{:.4f}".format(matrix_2[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_2[0,15]), "{:.4f}".format(matrix_2[0,31]))

print('\033[1m' + 'Absorbance FWHM (direct)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_3[0,4]), "{:.4f}".format(matrix_3[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_3[0,5]), "{:.4f}".format(matrix_3[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_3[0,6]), "{:.4f}".format(matrix_3[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_3[0,7]), "{:.4f}".format(matrix_3[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_3[0,12]), "{:.4f}".format(matrix_3[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_3[0,13]), "{:.4f}".format(matrix_3[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_3[0,14]), "{:.4f}".format(matrix_3[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_3[0,15]), "{:.4f}".format(matrix_3[0,31]))
print('\033[1m' + 'Absorbance FWHM (cascade)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_4[0,4]), "{:.4f}".format(matrix_4[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_4[0,5]), "{:.4f}".format(matrix_4[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_4[0,6]), "{:.4f}".format(matrix_4[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_4[0,7]), "{:.4f}".format(matrix_4[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_4[0,12]), "{:.4f}".format(matrix_4[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_4[0,13]), "{:.4f}".format(matrix_4[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_4[0,14]), "{:.4f}".format(matrix_4[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_4[0,15]), "{:.4f}".format(matrix_4[0,31]))

Model with Dropout at Inference
Absorbance Peak
MAE (eV): 	 0.0234 0.0035
R2 (eV):	 0.9604 0.0133
RAE (eV):	 0.1788 0.0290
RMSE (eV):	 0.0304 0.0047
MAE (nm):	 5.3099 0.7778
R2 (nm):	 0.9591 0.0120
RAE (nm):	 0.1811 0.0276
RMSE (nm):	 6.8720 0.9725
Absorbance FWHM (direct)
MAE (eV): 	 0.0107 0.0015
R2 (eV):	 0.4273 0.1385
RAE (eV):	 0.6618 0.0520
RMSE (eV):	 0.0157 0.0026
MAE (nm):	 2.6557 0.4292
R2 (nm):	 0.5443 0.1462
RAE (nm):	 0.5792 0.0575
RMSE (nm):	 3.9067 0.8099
Absorbance FWHM (cascade)
MAE (eV): 	 0.0087 0.0014
R2 (eV):	 0.5897 0.1066
RAE (eV):	 0.5374 0.0649
RMSE (eV):	 0.0132 0.0024
MAE (nm):	 2.1534 0.3407
R2 (nm):	 0.6931 0.0853
RAE (nm):	 0.4719 0.0610
RMSE (nm):	 3.1921 0.5740


In [12]:
np.savetxt("NN+MMCN_PL_Val_peak.csv", matrix, delimiter=",")
np.savetxt("NN+MMCN_PL_Val_fwhm.csv", matrix_fwhm, delimiter=",")
np.savetxt("NN+MMCN_PL_Val_cascade.csv", matrix_cascade, delimiter=",")